In [1]:
import json
import re
import math # for ceiling
import import_ipynb
from normalize_string import normalize_string
from typing import List, Dict, Union, Any

path_core = r'E:\Coinflip\Documents\Programmieren\LOR Card Bot\source\custom\core.json'
with open (path_core, encoding='utf8') as f:
        core = json.load(f)
path_cards = r'E:\Coinflip\Documents\Programmieren\LOR Card Bot\source\custom\cards.json'
with open (path_cards, encoding='utf8') as f:
        cards = json.load(f)
path_emotes = r'E:\Coinflip\Documents\Programmieren\LOR Card Bot\source\custom\emotes_catalog.json'
with open (path_emotes, encoding='utf8') as f:
        emotes = json.load(f)
path_emotes_nocatalog = r'E:\Coinflip\Documents\Programmieren\LOR Card Bot\source\custom\emotes.json'
with open (path_emotes_nocatalog, encoding='utf8') as f:
        emotes_nocatalog = json.load(f)
'''
returns List[List[keywordname, description], "core"]
'''
def all_core() -> List[Union[List[Union[str, str]], "core"]]:
    core_list = []
    for key in core['core']:
        d = {
            'name' : key,
            'description' : core['core'][key]
        }
        core_list.append([d, "core"])
    return core_list

'''
returns List[dict, "card"]
'''
def all_cards_full() -> List[Union[dict, "card"]]:
    card_list = []
    for key in cards['cards']:
        card_list.append([cards['cards'][key], "card"])
    return card_list

def all_cards_names() -> List[str]:
    card_list = []
    for key in cards['cards']:
        card_list.append(key)
    return card_list


'''
if a key "norm_context" is found in dict emotes or a nested dict catalog inside emotes, return value of key. 
Otherwise, return context
'''
def emote(context: str, catalog: Union[str, None], emotes = emotes) -> str:
    norm_context = normalize_string(context)
    print(norm_context)
    if catalog:
        if catalog in emotes:
            if norm_context in emotes[catalog]: 
                return emotes[catalog][norm_context]
    else:
        if norm_context in emotes:
            return emotes[norm_context]
    return context


'''
context_type = "core", "card"
returns List[context_dict, context_type]
'''
def strings_between_delimiters(user_input: str) -> List[str]:
    pattern = re.compile(r'\{\{[^\{\}]*\}\}')
    #\{\{ for beginning {{
    #[^\{\}]* for any number of symbols other than {,}
    #\}\} for ending }}
    
    matches = pattern.findall(user_input)
    found_strings = []
    for match in matches:
        found_strings.append(match[2:-2])  #slices {{}} out
    return found_strings

def find_context(user_input: str) -> List[Union[Dict, str]]:
    user_input = normalize_string(user_input)
    #core
    if user_input in core['lookup']:
        d = {
            "name" : core['lookup'][user_input],
            "description" : core['core'][core['lookup'][user_input]]      
        }
        return [d, "core"]
    #cards
    if user_input in cards['lookup']:
        return [cards['cards'][cards['lookup'][user_input]], "card"]
    
    #context not found
    print (f'Context {user_input} not found.')
    return [user_input, None]


'''
input: List[data, datatype]
data can be core, card
datatype can be "core", "card"
'''
def context_to_string(context: List[Union[Dict, str]]) ->str:
    d = context[0]
    if context[1] == "core":
        name = emote(d['name'],  None, emotes_nocatalog)
        description = d['description']
        return f"{name}: {description}"
    elif context[1] == "card":
        if d['type'] == "Spell":
            d['keywords'][:] = [x for x in d['keywords'] if x not in (("Slow", "Fast", "Burst"))] #remove slow,fast,burst out of keywordlist
            out =  (f"{d['cardname']}\n"
                    f"{emote(d['type'], 'type')}Cost:{str(d['cost'])}{d['spellSpeed']}{''.join([emote(kw, 'keywords') for kw in d['keywords']])}\n"
                    f"{emote(d['region'], 'region')}{emote(d['rarity'], 'rarity')} | {d['description']}")
        elif d['type'] == "Unit":
            if d['supertype'] == "Champion":
                out = (f"{d['cardname']}\n"
                    f"{emote('champion', 'type')}Cost:{str(d['cost'])} Attack:{str(d['attack'])} Health:{str(d['health'])}{''.join([emote(kw, 'keywords') for kw in d['keywords']])}{d['subtype']}\n"
                    f"{emote(d['region'], 'region')}{emote(d['rarity'], 'rarity')}{' | ' + d['description'] if d['description'] else ''}")
                if d['levelupDescription']:
                    out += f"\nLevel Up: {d['levelupDescription']}"
            else:
                out = (f"{d['cardname']}\n"
                    f"{emote(d['type'], 'type')}Cost:{str(d['cost'])} Attack:{str(d['attack'])} Health:{str(d['health'])}{''.join([emote(kw, 'keywords') for kw in d['keywords']])}{d['subtype']}\n"
                    f"{emote(d['region'], 'region')}{emote(d['rarity'], 'rarity')}{' | ' + d['description'] if d['description'] else ''}")
        return out
    else:
        print(f"{context[0]} not found")
        return ""

'''
input: List[str, str, ...]
returns: List[str, str, ...] with len(str) <= maxlength for every str
'''
def to_messages(strlist: Union[List[str], str], maxlength = 2000) -> List[str]:
    if isinstance is None:
        return [""]
    else:
        blocklist = []
        if isinstance(strlist, list):
            tmp = ""
            for elem in strlist:
                if len(elem) >= maxlength:
                    if tmp: 
                        blocklist.append(tmp)
                        tmp = ""
                    while len(elem) >= maxlength:
                        blocklist.append(elem[:maxlength])
                        elem = elem[maxlength:]
                    if elem: blocklist.append(elem)
                elif len(elem) + len(tmp) + len('\n') >= maxlength:
                    blocklist.append(tmp)
                    tmp = elem
                else:
                    if tmp:
                        tmp = tmp + '\n' + elem
                    else:
                        tmp = elem
            if tmp: blocklist.append(tmp)

        elif isinstance(strlist, str):
            while len(strlist) >= maxlength:
                blocklist.append(strlist[:maxlength])
                strlist = strlist[maxlength:]
            if strlist: 
                blocklist.append(strlist)

        return blocklist

def context_to_messages(context: Union[str, List[str]]) -> List[str]:
    if isinstance(context, str):
        a = find_context(context)
        b = context_to_string(a)
    elif isinstance(context, ((list,tuple))):
        b = []
        for elem in context:
            a = find_context(elem)
            b.append(context_to_string(a))
    else:
        return [f"unknown context type: {type(context)}"]
    return to_messages(b)
            
        
    
    

importing Jupyter notebook from normalize_string.ipynb


In [2]:
if __name__ == "__main__":
    print (context_to_messages("teemo"))
    
    i = 0
    for card in cards['cards']:
        print (cards['cards'][card])
        if cards['cards'][card]['collectible']:
            print (cards['cards'][card]['cardname'])
            i += 1

    print(i)

champion
elusive
piltoverzaun
champion
["Teemo\n<:champion_type:684155905679556630>Cost:1 Attack:1 Health:1<:elusive:684155906027814982>\n<:piltoverzaun:684155906497183753><:champion:684155905679556644> | Nexus Strike: Plant 5 Poison Puffcaps on random cards in the enemy deck.\nLevel Up: You've planted 15+ Poison Puffcaps."]
